# Analysis

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [2]:
# set up plotting settings for dark mode.
from jupyterthemes import jtplot
jtplot.style(theme="grade3", context="notebook", ticks=True, grid=False)
from src.plot_settings import ps_defaults
ps_defaults(use_tex=False, dpi=150)

In [3]:
import os
from typing import Callable, Union
import inspect
import time
import xarray as xr
import wandb
import hydra
from omegaconf import DictConfig, OmegaConf
from src.constants import (
    FIGURE_PATH,
    OCEAN_DATA_PATH,
    OCEAN_OUTPUT_PATH,
    SRC_PATH,
    OCEAN_RUN_PATH,
    OCEAN_SRC_PATH,
    PROJECT_PATH,
)
from src.visualisation.ani import animate_xr_da
from src.utils import timeit, fix_calendar

In [14]:
def fix_calendar_old(
    xr_in: Union[xr.Dataset, xr.DataArray], timevar: str = "T"
) -> Union[xr.Dataset, xr.DataArray]:
    """Fix and decode the calendar.

    Args:
        xr_in (Union[xr.Dataset, xr.DataArray]): the xarray object input
        timevar (str, optional): The time variable name. Defaults to "T".

    Returns:
        Union[xr.Dataset, xr.DataArray]: same type and xr_in with fixed calendar.

    """
    # make all into dataset
    if isinstance(xr_in, xr.DataArray):
        ds = xr_in.to_dataset()
    else:
        ds = xr_in

    # add 360_day attribute
    if "calendar" not in ds[timevar].attrs:
        ds[timevar].attrs["calendar"] = "360_day"
    else:
        if ds[timevar].attrs["calendar"] == "360":
            ds[timevar].attrs["calendar"] = "360_day"

    # decode
    ds = xr.decode_cf(ds)
    """
    ds.assign(
        timevar=np.array(
            [pd.to_datetime(x.strftime()).to_datetime64() for x in ds[timevar].values, 
             dtype=np.datetime64
        )
    )
    """
    """"
    ds.assign_coords(
        "Q" timevar=np.array(
            [pd.to_datetime(x.strftime()).to_datetime64() for x in ds[timevar].values],
            dtype=np.datetime64
        )
    )
    """
    print(ds)
    # ds[timevar].values = np.array([pd.to_datetime(x.strftime()).to_datetime64() for x in ds[timevar].values])

    # ds[timevar].values = ds[timevar].values.to_datetime()

    # transform back into original type
    if isinstance(xr_in, xr.DataArray):
        xr_out = ds.to_array()
    else:
        xr_out = ds

    return xr_out


def rdict(index: int) -> dict:
    """Returns renaming dict for xarray.DataArray.

    Args:
        index (int): index on coords

    Returns:
        dict: renaming dict.
    """
    return {
        "T_0" + str(index): "time",
        "Y_0" + str(index): "y",
        "X_0" + str(index): "x",
        "L_0" + str(index): "Z",
    }


@timeit
def animate_all() -> None:
    """Animate the sst into gifs."""
    for x in [
        "om_diag",
        # "om_run2f",
        # "qflx",
        # "qflx-0",
    ]:
        animate_ds(
            xr.open_dataset(
                str(os.path.join(OCEAN_OUTPUT_PATH, x)) + ".nc", decode_times=False
            ),
            x,
        )


@timeit
def animate_ds(ds: xr.Dataset, file_name: str) -> None:
    """Animate the `xarray.Dataset`."""
    cmap_d = {
        "DYN_PRES": "delta",
        "SST_QFLX": "delta",
        "SST_SST": "sst",
        "qflx": "delta",
        "QFLX": "delta",
        "TDEEP_HTHERM": "sst",
        "TDEEP_TDEEP": "sst",
        "TDEEP_HMODEL": "sst",
    }
    unit_d = {"SST_SST": "$^{\circ}$C"}
    for y in ds.variables:
        y = str(y)
        if "X_" not in y:
            if "Y_" not in y:
                if "L_" not in y:
                    if "T_" not in y or "SST" in y:
                        if "GRID" != y:
                            print(y)
                            da = ds[y]
                            for key in da.coords:
                                num = str(key)[3]
                            da = da.rename(rdict(num))
                            if y in unit_d:
                                da.attrs["units"] = unit_d[y]
                            da.coords["x"].attrs["units"] = "$^{\circ}$E"
                            da.coords["y"].attrs["units"] = "$^{\circ}$N"
                            da = da.where(da != 0.0).isel(Z=0)
                            da = fix_calendar(da, timevar="time")
                            if "variable" in da.dims:
                                da = da.isel(variable=0)
                            da = da.rename(y)
                            if y in unit_d:
                                da.attrs["units"] = unit_d[y]
                            da.attrs["long_name"] = y
                            da.attrs["name"] = y
                            print(da)
                            animate_xr_da(
                                da,
                                video_path=file_name + "_" + y + ".gif",
                                vcmap=cmap_d[y],
                            )

In [15]:
animate_all()

om_diag_SST_SST.gif:   0%|          | 0/25 [00:00<?, ?it/s]

SST_SST
<xarray.DataArray 'SST_SST' (time: 25, y: 161, x: 360)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [-0.24075225,  0.01570407,  0.26875567, ..., -1.0816206 ,
         -0.7841495 , -0.50387925],
        [-0.8261021 , -0.55967414, -0.29426387, ..., -1.5676941 ,
         -1.3554841 , -1.0942034 ],
        [-1.4988613 , -1.3286685 , -1.1596736 , ..., -1.8076593 ,
         -1.7545118 , -1.6456904 ]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...

om_diag_SST_QFLX.gif:   0%|          | 0/25 [00:00<?, ?it/s]

Video om_diag_SST_SST.gif made.
'animate_xr_da'  13.06881 s

SST_QFLX
<xarray.DataArray 'SST_QFLX' (time: 25, y: 161, x: 360)>
array([[[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        ...,
        [-3.1415999e-04,  2.1026271e-05,  3.5069848e-04, ...,
         -1.4108991e-03, -1.0238644e-03, -6.5546297e-04],
        [-1.0779888e-03, -7.2997710e-04, -3.8398404e-04, ...,
         -2.0468503e-03, -1.7698468e-03, -1.4266199e-03],
        [-1.9558789e-03, -1.7329683e-03, -1.5132546e-03, ...,
         -2.3608049e-03, -2.2904719e-03, -2.1469037e-03]],

       [[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [         

om_diag_DYN_PRES.gif:   0%|          | 0/25 [00:00<?, ?it/s]

...

Video om_diag_DYN_PRES.gif made.
'animate_xr_da'  12.90486 s

TDEEP_HMODEL
<xarray.DataArray 'TDEEP_HMODEL' (time: 25, y: 161, x: 360)>
array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [196.78485, 195.65393, 195.92242, ..., 195.65393, 195.92242,
         197.60728],
        [197.7229 , 197.84193, 198.24216, ..., 197.84193, 198.24216,
         198.91867],
        [198.97867, 199.40132, 199.88469, ..., 199.40132, 199.88469,
         200.42615]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
...
        [196.05696, 195.1771 , 195.

om_diag_TDEEP_TDEEP.gif:   0%|          | 0/25 [00:00<?, ?it/s]

Video om_diag_TDEEP_HMODEL.gif made.
'animate_xr_da'  12.92034 s

TDEEP_TDEEP
<xarray.DataArray 'TDEEP_TDEEP' (time: 25, y: 161, x: 360)>
array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [28.786192, 28.728796, 28.74236 , ..., 28.728796, 28.74236 ,
         28.828375],
        [28.834335, 28.840479, 28.861185, ..., 28.840479, 28.861185,
         28.89611 ],
        [28.899189, 28.920721, 28.944891, ..., 28.920721, 28.944891,
         28.971151]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
...
        [28.74917 , 28.704803, 28

om_diag_TDEEP_TDEEP.gif: 100%|██████████| 25/25 [00:08<00:00,  3.02it/s]


Video om_diag_TDEEP_TDEEP.gif made.
'animate_xr_da'  10.68659 s

'animate_ds'  63.35162 s

'animate_all'  63.37951 s



In [18]:
xr.open_dataset(
    str(os.path.join(OCEAN_DATA_PATH, "qflx-0")) + ".nc", decode_times=False
).rename(
    {
        "T": "time",
        "Y": "y",
        "X": "x",
        "Z": "Z",
    }
)

<xarray.Dataset>
Dimensions:  (Z: 1, time: 12, x: 360, y: 181)
Coordinates:
  * x        (x) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * y        (y) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * Z        (Z) float32 1.0
  * time     (time) float32 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5 10.5 11.5
Data variables:
    qflx     (time, Z, y, x) float32 ...

In [25]:
fix_calendar(
    xr.open_dataset(
        str(os.path.join(OCEAN_OUTPUT_PATH, "om_diag")) + ".nc", decode_times=False
    ).DYN_PRES,
    timevar="T_02",
).T_02.values[
    0
]  # ._to_real_datetime()

cftime.Datetime360Day(1955, 12, 30, 10, 59, 51, 650391)

In [26]:
fix_calendar(
    xr.open_dataset(
        str(os.path.join(OCEAN_OUTPUT_PATH, "om_diag")) + ".nc", decode_times=False
    ).DYN_PRES,
    timevar="T_02",
).T_02.values[0]._to_real_datetime()

real_datetime(1955, 12, 30, 10, 59, 51, 650391)

In [34]:
pd.to_datetime(
    fix_calendar(
        xr.open_dataset(
            str(os.path.join(OCEAN_OUTPUT_PATH, "om_diag")) + ".nc", decode_times=False
        ).DYN_PRES,
        timevar="T_02",
    )
    .T_02.values[0]
    .strftime()[0:10]
)

Timestamp('1955-12-30 00:00:00')

In [33]:
import pandas as pd

In [61]:
pd.to_datetime(
    fix_calendar(
        xr.open_dataset(
            str(os.path.join(OCEAN_OUTPUT_PATH, "om_diag")) + ".nc", decode_times=False
        ).DYN_PRES,
        timevar="T_02",
    )
    .T_02.values[0]
    .strftime()
).to_datetime64()

<xarray.Dataset>
Dimensions:   (L_02: 1, T_02: 25, X_02: 360, Y_02: 161)
Coordinates:
  * X_02      (X_02) float32 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * Y_02      (Y_02) float32 -80.0 -79.0 -78.0 -77.0 ... 77.0 78.0 79.0 80.0
  * L_02      (L_02) float32 0.0
  * T_02      (T_02) object 1955-12-30 10:59:51.650391 ... 1957-12-28 15:56:5...
Data variables:
    DYN_PRES  (T_02, L_02, Y_02, X_02) float32 ...


Timestamp('1955-12-30 10:59:51')